In [ ]:
# Iterative RAFFLE structure search
from ase.io import read
from ase import Atoms
from raffle.generator import raffle_generator
from mace.calculators import mace_mp
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
generator = raffle_generator()
generator.distributions.set_history_len(10)
calc = mace_mp(model="medium", dispersion=False, default_dtype="float32", device='cpu')

In [ ]:
host = Atoms('C', positions=[(0, 0, 0)], cell=[10, 10, 10], pbc=True)
host.calc = calc
generator.set_host(host)

In [ ]:
aa_stack = read("POSCAR_AA_stack")
ab_stack = read("POSCAR_AB_stack")
aabbcc_stack = read("POSCAR_AABBCC_stack")
aba_stack = read("POSCAR_ABA_stack")
abab_stack = read("POSCAR_ABAB_stack")
lonsdaleite = read("POSCAR_lonsdaleite")
diamond = read("POSCAR_diamond")

# 0 = Unknown mp entry
# 1 = mp-169 = 0.001 OR mp-3347313 = 0.000
# 2 = mp-2516584 = 0.002
# 3 = mp-606949 = 0.006
# 4 = mp-569416 = 0.002
# 5 = mp-47 = 0.139
# 6 = mp-66 = 0.112

In [ ]:
aa_stack.calc = calc
ab_stack.calc = calc
aabbcc_stack.calc = calc
aba_stack.calc = calc
abab_stack.calc = calc
lonsdaleite.calc = calc
diamond.calc = calc

In [ ]:
generator.distributions.set_element_energies(
    {
        "C": ab_stack.get_potential_energy() / len(ab_stack),
    }
)

In [ ]:
database = [
    aa_stack,
    ab_stack,
    aabbcc_stack,
    aba_stack,
    abab_stack,
    lonsdaleite,
    diamond,
]

In [ ]:
### Optional parameters
generator.distributions.set_kBT(0.2)
# generator.distributions.set_width([0.02, np.pi/200.0, np.pi/200.0])
# generator.distributions.set_cutoff_min([0.5, 0.0, 0.0])
# generator.distributions.set_cutoff_max([6.0, np.pi, np.pi])


In [ ]:
### Explore the effect of excluding 3- and 4-body terms
# generator.distributions.set_radius_distance_tol([0.0, 0.0, 0.0, 0.0])
# generator.distributions.set_radius_distance_tol([1.5, 2.5, 0.0, 0.0])
# generator.distributions.set_radius_distance_tol([1.5, 2.5, 3.0, 6.0])

In [ ]:
generator.distributions.create(database)

In [ ]:
descriptor = generator.get_descriptor()

In [ ]:
# Create a figure with 3 subplots side by side
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot for each n-body descriptor (2-body, 3-body, 4-body)
for j in range(3):
    # Calculate x-axis values
    x = np.arange(generator.distributions.cutoff_min[j],
                generator.distributions.cutoff_max[j] + generator.distributions.width[j],
                generator.distributions.width[j])

    # Plot on the respective subplot
    for idx in range(len(descriptor[j])):
        axes[j].plot(x, descriptor[j][idx,:])

    # Set labels and title for each subplot
    axes[j].set_ylabel('Descriptor value')
    axes[j].set_title(f'{j+2}-body descriptor')

axes[0].set_xlabel('Distance (Å)')
axes[1].set_xlabel('3-body angle (radians)')
axes[2].set_xlabel('Improper dihedral angle (radians)')
plt.tight_layout()
plt.show()

In [ ]:
for i, structure in enumerate(database):
    print("energy", structure.get_potential_energy()/len(structure) - ab_stack.get_potential_energy()/len(ab_stack))
    print("viability", generator.evaluate(structure))
    print()